<a href="https://colab.research.google.com/github/345bc/TriTueNhanTao/blob/main/BaiTapTuan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Thuật toán Minimax và Cắt tỉa Alpha-Beta (Alpha-Beta Pruning)**

Đây là hai thuật toán cốt lõi trong trí tuệ nhân tạo, đặc biệt dùng để lập chiến lược cho các trò chơi hai người đối kháng có thông tin hoàn hảo (perfect information) như Cờ Vua, Cờ Tướng, Cờ Caro, Othello, Connect Four…
1. Thuật toán Minimax
Ý tưởng chính:

Một người chơi là Max (muốn tối đa hóa điểm số).
Người chơi còn lại là Min (muốn tối thiểu hóa điểm số của Max).
Cây trò chơi (game tree): mỗi nút là một trạng thái bàn cờ, mỗi cạnh là một nước đi hợp lệ.
Minimax duyệt toàn bộ cây đến độ sâu cho trước hoặc đến trạng thái kết thúc, rồi tính giá trị tốt nhất mà Max có thể đạt được khi cả hai bên đều chơi tối ưu.

Cách hoạt động (đệ quy):


    if maximizingPlayer (lượt Max):
        maxEval = -∞
        for each child in node:
            eval = minimax(child, depth-1, FALSE)
            maxEval = max(maxEval, eval)
        return maxEval

    else (lượt Min):
        minEval = +∞
        for each child in node:
            eval = minimax(child, depth-1, TRUE)
            minEval = min(minEval, eval)
        return minEval

Kết quả: tại gốc cây, giá trị trả về chính là nước đi tốt nhất cho người đang đi (nếu là Max thì chọn con có giá trị lớn nhất, nếu là Min thì chọn con có giá trị nhỏ nhất).
Nhược điểm lớn: Độ phức tạp thời gian là O(b^d) với b = branching factor (số nước đi trung bình), d = độ sâu. Với cờ vua b≈35, d=6 đã là hàng triệu nút → rất chậm.
2. Cắt tỉa Alpha-Beta (Alpha-Beta Pruning)
Là phiên bản tối ưu của minimax, có thể cắt bỏ rất nhiều nhánh mà không ảnh hưởng đến kết quả cuối cùng.
Thêm 2 tham số:

α (alpha): giá trị tốt nhất mà Max đã chắc chắn đạt được ở các nhánh trước đó.
β (beta): giá trị tốt nhất mà Min đã chắc chắn ép Max xuống ở các nhánh trước đó.

Quy tắc cắt tỉa:

Nếu một nhánh cho Min tìm được giá trị ≤ α → Max sẽ không chọn nhánh cha này → có thể bỏ qua các nước đi còn lại (beta cutoff).
Nếu một nhánh cho Max tìm được giá trị ≥ β → Min sẽ không cho Max đi nhánh này → bỏ qua các nước đi còn lại (alpha cutoff).

Code Alpha-Beta:
pseudocodealphabeta(node, depth, α, β, maximizingPlayer):
    if depth == 0 or node kết thúc:
        return đánh giá(node)

    if maximizingPlayer:
        maxEval = -∞
        for each child in node:
            eval = alphabeta(child, depth-1, α, β, FALSE)
            maxEval = max(maxEval, eval)
            α = max(α, maxEval)
            if β ≤ α:               // Beta cutoff
                break
        return maxEval
    else:
        minEval = +∞
        for each child in node:
            eval = alphabeta(child, depth-1, α, β, TRUE)
            minEval = min(minEval, eval)
            β = min(β, minEval)
            if β ≤ α:               // Alpha cutoff
                break
        return minEval
Gọi ban đầu:
pseudocodealphabeta(root, depth, -∞, +∞, TRUE)   // TRUE nếu hiện tại là lượt Max
Hiệu quả thực tế:

Trong trường hợp lý tưởng (thứ tự nước đi tốt nhất được xét trước): chỉ cần duyệt O(b^{d/2}) nút → nhanh hơn rất nhiều (căn bậc hai).
Thực tế thường đạt 5-20 lần nhanh hơn minimax thuần tùy trò chơi và cách sắp xếp nước đi.

3. Các cải tiến thường dùng kèm Alpha-Beta

Sắp xếp nước đi (Move Ordering)
Đưa các nước đi có vẻ tốt nhất lên đầu → tăng số lần cắt tỉa.
Thường dùng: nước ăn quân, lịch sử heuristic, killer heuristic, MVV-LVA (cờ vua).

Bảng chuyển vị (Transposition Table)
Lưu kết quả đã tính cho các vị trí đã gặp (dù đến bằng đường khác nhau).
Iterative Deepening
Tăng dần độ sâu, dùng kết quả độ sâu nông để sắp xếp nước đi cho độ sâu sâu hơn.
Quiescence Search
Khi đến giới hạn độ sâu, tiếp tục tìm kiếm thêm các nước ăn quân để tránh lỗi “horizon effect”.
Aspiration Windows, PVS, MTD(f)
Các kỹ thuật nâng cao để giảm số nút còn hơn nữa.



Triển khai trò chơi Caro

In [ ]:
import pygame
import sys
import math
import copy

BOARD_SIZE = 600
SIDEBAR_WIDTH = 250
WIDTH = BOARD_SIZE + SIDEBAR_WIDTH
HEIGHT = 600

ROWS = 15
COLS = 15
SQUARE_SIZE = BOARD_SIZE // COLS
CIRCLE_RADIUS = SQUARE_SIZE // 3
LINE_WIDTH = 2
piece_width = 4

C_BG_MAIN = (250, 248, 239)
C_BG_SIDEBAR = (44, 62, 80)
C_GRID = (189, 195, 199)
C_X = (46, 204, 113)
C_O = (231, 76, 60)
C_TEXT_LIGHT = (236, 240, 241)
C_TEXT_DARK = (44, 62, 80)
C_HIGHLIGHT = (241, 196, 15)
C_HOVER = (200, 200, 200)

PLAYER = 1
AI = 2
EMPTY = 0

pygame.init()
FONT_BIG = pygame.font.SysFont('arial', 40, bold=True)
FONT_SMALL = pygame.font.SysFont('arial', 20)
FONT_BTN = pygame.font.SysFont('arial', 25, bold=True)

class Board:
    def __init__(self):
        self.squares = [[EMPTY] * COLS for _ in range(ROWS)]
        self.marked_sqrs = 0
        self.last_move = None

    def mark_sqr(self, row, col, player):
        self.squares[row][col] = player
        self.marked_sqrs += 1
        self.last_move = (row, col)

    def empty_sqr(self, row, col):
        return self.squares[row][col] == EMPTY

    def get_empty_sqrs(self):
        moves = set()
        for r in range(ROWS):
            for c in range(COLS):
                if self.squares[r][c] != EMPTY:
                    for dr in [-1, 0, 1]:
                        for dc in [-1, 0, 1]:
                            nr, nc = r + dr, c + dc
                            if 0 <= nr < ROWS and 0 <= nc < COLS and self.squares[nr][nc] == EMPTY:
                                moves.add((nr, nc))
        if not moves and self.marked_sqrs == 0: return [(ROWS//2, COLS//2)]
        return list(moves)

    def is_full(self):
        return self.marked_sqrs == ROWS * COLS

    def check_win(self, player):
        for r in range(ROWS):
            for c in range(COLS):
                if self.squares[r][c] == player:
                    if c + 4 < COLS and all(self.squares[r][c+i] == player for i in range(5)): return True
                    if r + 4 < ROWS and all(self.squares[r+i][c] == player for i in range(5)): return True
                    if r + 4 < ROWS and c + 4 < COLS and all(self.squares[r+i][c+i] == player for i in range(5)): return True
                    if r + 4 < ROWS and c - 4 >= 0 and all(self.squares[r+i][c-i] == player for i in range(5)): return True
        return False

class AI_Engine:
    def __init__(self, level=1, player=AI):
        self.level = level
        self.player = player

    def evaluate_position(self, board, player):
        score = 0
        opponent = PLAYER if player == AI else AI
        for r in range(ROWS):
            for c in range(COLS):
                if board.squares[r][c] == player: score += self.check_pattern(board, r, c, player)
                elif board.squares[r][c] == opponent: score -= self.check_pattern(board, r, c, opponent) * 1.2
        return score

    def check_pattern(self, board, r, c, player):
        score = 0
        directions = [(0,1), (1,0), (1,1), (1,-1)]
        for dr, dc in directions:
            count = 0
            blocked = 0
            for i in range(1, 5):
                nr, nc = r + dr*i, c + dc*i
                if 0 <= nr < ROWS and 0 <= nc < COLS:
                    if board.squares[nr][nc] == player: count += 1
                    elif board.squares[nr][nc] != EMPTY:
                        blocked += 1
                        break
                else:
                    blocked += 1
                    break
            if count == 4: score += 100000
            elif count == 3: score += 5000 if blocked == 0 else 500
            elif count == 2: score += 500 if blocked == 0 else 50
            elif count == 1: score += 10
        return score

    def minimax(self, board, depth, alpha, beta, maximizing):
        if board.check_win(AI): return 10000000, None
        if board.check_win(PLAYER): return -10000000, None
        if board.is_full() or depth == 0: return self.evaluate_position(board, AI), None

        possible_moves = board.get_empty_sqrs()
        possible_moves.sort(key=lambda x: (x[0]-ROWS//2)**2 + (x[1]-COLS//2)**2)

        best_move = None
        if maximizing:
            max_eval = -math.inf
            for (row, col) in possible_moves:
                temp_board = copy.deepcopy(board)
                temp_board.mark_sqr(row, col, AI)
                eval_score, _ = self.minimax(temp_board, depth-1, alpha, beta, False)
                if eval_score > max_eval:
                    max_eval = eval_score
                    best_move = (row, col)
                alpha = max(alpha, eval_score)
                if beta <= alpha: break
            return max_eval, best_move
        else:
            min_eval = math.inf
            for (row, col) in possible_moves:
                temp_board = copy.deepcopy(board)
                temp_board.mark_sqr(row, col, PLAYER)
                eval_score, _ = self.minimax(temp_board, depth-1, alpha, beta, True)
                if eval_score < min_eval:
                    min_eval = eval_score
                    best_move = (row, col)
                beta = min(beta, eval_score)
                if beta <= alpha: break
            return min_eval, best_move

class Game:
    def __init__(self):
        self.board = Board()
        self.ai = AI_Engine(level=2)
        self.player = PLAYER
        self.gamemode = 'ai'
        self.running = True
        self.winner = None

        self.reset_btn_rect = pygame.Rect(BOARD_SIZE + 25, 500, 200, 50)

    def draw_board_grid(self, screen):
        screen.fill(C_BG_MAIN)
        for r in range(ROWS):
            pygame.draw.line(screen, C_GRID, (0, r * SQUARE_SIZE), (BOARD_SIZE, r * SQUARE_SIZE), LINE_WIDTH)
        for c in range(COLS):
            pygame.draw.line(screen, C_GRID, (c * SQUARE_SIZE, 0), (c * SQUARE_SIZE, HEIGHT), LINE_WIDTH)

    def draw_sidebar(self, screen):
        pygame.draw.rect(screen, C_BG_SIDEBAR, (BOARD_SIZE, 0, SIDEBAR_WIDTH, HEIGHT))

        title_surf = FONT_BIG.render("CARO AI", True, C_TEXT_LIGHT)
        screen.blit(title_surf, (BOARD_SIZE + 40, 30))

        status_text = "Your Turn (X)" if self.player == PLAYER else "AI Thinking..."
        if not self.running:
            if self.winner == PLAYER: status_text = "YOU WIN!"
            elif self.winner == AI: status_text = "AI WINS!"
            elif self.winner == 'Draw': status_text = "DRAW!"

        status_color = C_X if self.player == PLAYER else C_O
        if not self.running: status_color = C_HIGHLIGHT

        status_surf = FONT_SMALL.render(status_text, True, status_color)
        screen.blit(status_surf, (BOARD_SIZE + 25, 100))

        info_surf1 = FONT_SMALL.render("Player: Green (X)", True, C_TEXT_LIGHT)
        info_surf2 = FONT_SMALL.render("AI: Red (O)", True, C_TEXT_LIGHT)
        screen.blit(info_surf1, (BOARD_SIZE + 25, 150))
        screen.blit(info_surf2, (BOARD_SIZE + 25, 180))

        pygame.draw.rect(screen, C_HIGHLIGHT, self.reset_btn_rect, border_radius=10)
        btn_text = FONT_BTN.render("NEW GAME", True, C_BG_SIDEBAR)
        text_rect = btn_text.get_rect(center=self.reset_btn_rect.center)
        screen.blit(btn_text, text_rect)

    def draw_pieces(self, screen):
        if self.board.last_move:
            row, col = self.board.last_move
            x = col * SQUARE_SIZE
            y = row * SQUARE_SIZE
            s = pygame.Surface((SQUARE_SIZE, SQUARE_SIZE))
            s.set_alpha(100)
            s.fill(C_HIGHLIGHT)
            screen.blit(s, (x, y))

        for row in range(ROWS):
            for col in range(COLS):
                if self.board.squares[row][col] != EMPTY:
                    center_x = col * SQUARE_SIZE + SQUARE_SIZE // 2
                    center_y = row * SQUARE_SIZE + SQUARE_SIZE // 2

                    if self.board.squares[row][col] == PLAYER:
                        start1 = (center_x - SQUARE_SIZE//3, center_y - SQUARE_SIZE//3)
                        end1 = (center_x + SQUARE_SIZE//3, center_y + SQUARE_SIZE//3)
                        start2 = (center_x - SQUARE_SIZE//3, center_y + SQUARE_SIZE//3)
                        end2 = (center_x + SQUARE_SIZE//3, center_y - SQUARE_SIZE//3)
                        pygame.draw.line(screen, C_X, start1, end1, piece_width)
                        pygame.draw.line(screen, C_X, start2, end2, piece_width)

                    elif self.board.squares[row][col] == AI:
                        pygame.draw.circle(screen, C_O, (center_x, center_y), CIRCLE_RADIUS, piece_width)
                        pygame.draw.circle(screen, C_O, (center_x, center_y), CIRCLE_RADIUS//3, 1)

    def draw_hover(self, screen, pos):
        if pos[0] < BOARD_SIZE and self.running:
            col = pos[0] // SQUARE_SIZE
            row = pos[1] // SQUARE_SIZE
            if self.board.empty_sqr(row, col):
                center_x = col * SQUARE_SIZE + SQUARE_SIZE // 2
                center_y = row * SQUARE_SIZE + SQUARE_SIZE // 2
                s = pygame.Surface((SQUARE_SIZE, SQUARE_SIZE))
                s.set_alpha(50)
                s.fill(C_X if self.player == PLAYER else C_O)
                screen.blit(s, (col * SQUARE_SIZE, row * SQUARE_SIZE))

    def update(self, screen):
        self.draw_board_grid(screen)
        self.draw_pieces(screen)

        mouse_pos = pygame.mouse.get_pos()
        self.draw_hover(screen, mouse_pos)

        self.draw_sidebar(screen)
        pygame.display.update()

    def make_move(self, row, col):
        self.board.mark_sqr(row, col, self.player)
        self.next_turn()

    def next_turn(self):
        self.player = self.player % 2 + 1

    def check_game_over(self):
        if self.board.check_win(PLAYER):
            self.winner = PLAYER
            self.running = False
        elif self.board.check_win(AI):
            self.winner = AI
            self.running = False
        elif self.board.is_full():
            self.winner = 'Draw'
            self.running = False

    def reset(self):
        self.__init__()

def main():
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption('Caro AI - Modern UI')
    game = Game()

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

            if event.type == pygame.MOUSEBUTTONDOWN:
                pos = event.pos

                if game.reset_btn_rect.collidepoint(pos):
                    game.reset()
                    continue

                if pos[0] < BOARD_SIZE and game.running and game.player == PLAYER:
                    row = pos[1] // SQUARE_SIZE
                    col = pos[0] // SQUARE_SIZE

                    if game.board.empty_sqr(row, col):
                        game.make_move(row, col)
                        game.check_game_over()
                        game.update(screen)

        game.update(screen)

        if game.running and game.player == AI:
            pygame.time.wait(100)

            value, move = game.ai.minimax(game.board, game.ai.level, -math.inf, math.inf, True)

            if move:
                row, col = move
                game.make_move(row, col)
                game.check_game_over()
            else:
                game.running = False
                game.winner = 'Draw'

if __name__ == '__main__':
    main()

1. Hàm Đánh Giá (Heuristic Evaluation)
Đây là "đôi mắt" của AI. Vì không thể đi hết mọi nước đi đến tận cùng ván đấu, AI cần một cách để nhìn vào bàn cờ hiện tại và ước lượng xem mình đang thắng thế hay thua thế.

Code:
Python

    def evaluate_position(self, board, player):
        score = 0
        opponent = PLAYER if player == AI else AI
        for r in range(ROWS):
            for c in range(COLS):
                if board.squares[r][c] == player:
                    score += self.check_pattern(board, r, c, player)
                elif board.squares[r][c] == opponent:
                    # * 1.2 nghĩa là AI ưu tiên phòng thủ hơn tấn công một chút
                    score -= self.check_pattern(board, r, c, opponent) * 1.2
        return score

    def check_pattern(self, board, r, c, player):
        score = 0
        directions = [(0,1), (1,0), (1,1), (1,-1)] # Ngang, Dọc, Chéo chính, Chéo phụ
        for dr, dc in directions:
            count = 0
            blocked = 0
            # Kiểm tra 4 ô tiếp theo theo hướng đó
            for i in range(1, 5):
                nr, nc = r + dr*i, c + dc*i
                if 0 <= nr < ROWS and 0 <= nc < COLS:
                    if board.squares[nr][nc] == player: count += 1
                    elif board.squares[nr][nc] != EMPTY:
                        blocked += 1
                        break
                else:
                    blocked += 1 # Ra khỏi bàn cờ coi như bị chặn
                    break
            
            # Tính điểm dựa trên số quân liên tiếp và việc có bị chặn đầu hay không
            if count == 4: score += 100000       # 5 con thẳng hàng (thắng chắc)
            elif count == 3: score += 5000 if blocked == 0 else 500 # 4 con
            elif count == 2: score += 500 if blocked == 0 else 50   # 3 con
            elif count == 1: score += 10
        return score
Giải thích:
evaluate_position: Hàm này quét toàn bộ bàn cờ.

Nếu gặp quân của AI: Cộng điểm (Tấn công).

Nếu gặp quân của Người chơi: Trừ điểm (Phòng thủ).

Lưu ý * 1.2: Bạn đang nhân điểm của đối thủ với 1.2. Điều này làm cho điểm số âm (nguy hiểm) lớn hơn điểm số dương (lợi thế). Kết quả là AI sẽ có xu hướng "sợ thua" hơn là "ham thắng", dẫn đến lối chơi phòng thủ chặt chẽ hơn.

check_pattern: Hàm này kiểm tra 4 hướng từ một quân cờ cụ thể (Ngang, Dọc, 2 đường Chéo).

Nó đếm số quân liên tiếp (count) và xem hướng đó có bị chặn (blocked) bởi quân đối phương hoặc mép bàn cờ hay không.

Logic chấm điểm:

Một chuỗi X X X X (count=4) cực kỳ giá trị (100,000 điểm) vì nước tiếp theo sẽ thắng.

Một chuỗi _ X X X _ (count=3, không bị chặn) có giá trị cao (5,000 điểm) vì tạo thành nước đôi.

Một chuỗi O X X X _ (count=3, bị chặn 1 đầu) bị giảm giá trị mạnh (chỉ còn 500 điểm) vì đối thủ có thể chặn đầu còn lại dễ dàng.

2. Thuật toán Minimax & Cắt tỉa Alpha-Beta
Đây là "bộ não" tính toán nước đi.

Code:
Python

    def minimax(self, board, depth, alpha, beta, maximizing):
        # 1. Điều kiện dừng (Base case)
        if board.check_win(AI): return 10000000, None
        if board.check_win(PLAYER): return -10000000, None
        if board.is_full() or depth == 0: return self.evaluate_position(board, AI), None

        # 2. Lấy các nước đi khả thi và Sắp xếp tối ưu (Optimization)
        possible_moves = board.get_empty_sqrs()
        # Sắp xếp để ưu tiên các ô gần trung tâm trước -> Cắt tỉa Alpha-Beta hiệu quả hơn
        possible_moves.sort(key=lambda x: (x[0]-ROWS//2)**2 + (x[1]-COLS//2)**2)

        best_move = None
        
        # 3. Nhánh Tối đa hóa (Lượt của AI)
        if maximizing:
            max_eval = -math.inf
            for (row, col) in possible_moves:
                temp_board = copy.deepcopy(board)
                temp_board.mark_sqr(row, col, AI)
                # Đệ quy: Gọi lại minimax nhưng đổi lượt sang maximizing=False
                eval_score, _ = self.minimax(temp_board, depth-1, alpha, beta, False)
                
                if eval_score > max_eval:
                    max_eval = eval_score
                    best_move = (row, col)
                
                # Cắt tỉa Alpha
                alpha = max(alpha, eval_score)
                if beta <= alpha: break # Cắt tỉa: Không cần xét các nhánh còn lại
            return max_eval, best_move
            
        # 4. Nhánh Tối thiểu hóa (Lượt của Người chơi)
        else:
            min_eval = math.inf
            for (row, col) in possible_moves:
                temp_board = copy.deepcopy(board)
                temp_board.mark_sqr(row, col, PLAYER)
                # Đệ quy
                eval_score, _ = self.minimax(temp_board, depth-1, alpha, beta, True)
                
                if eval_score < min_eval:
                    min_eval = eval_score
                    best_move = (row, col)
                
                # Cắt tỉa Beta
                beta = min(beta, eval_score)
                if beta <= alpha: break # Cắt tỉa
            return min_eval, best_move
Giải thích:
Mục tiêu: AI giả lập việc đi thử các nước đi trong đầu. "Nếu mình đi ô A, đối thủ sẽ đi ô B, rồi mình đi ô C... thì kết cục thế nào?".

Minimax:

Maximizing (AI): Luôn chọn nước đi có điểm số cao nhất (max_eval).

Minimizing (Người chơi): AI giả định rằng người chơi rất thông minh và sẽ luôn chọn nước đi làm cho điểm số của AI thấp nhất (min_eval).

Cắt tỉa Alpha-Beta (alpha, beta):

Đây là kỹ thuật tối ưu hóa để AI chạy nhanh hơn.

Ví dụ: AI tìm thấy một nước đi A rất tốt. Sau đó AI xét nước đi B, nhưng mới xét được nửa chừng đã thấy nước đi B dẫn đến kết quả tệ hơn A. AI sẽ dừng ngay lập tức (break) việc tính toán tiếp các trường hợp con của B. Đây gọi là "cắt tỉa".

Tối ưu hóa sắp xếp (possible_moves.sort):

Dòng lệnh possible_moves.sort(key=lambda x: ...) sắp xếp các nước đi từ trung tâm bàn cờ ra ngoài.

Trong cờ Caro, các nước đi ở giữa thường mạnh hơn. Việc xét các nước đi mạnh trước giúp kỹ thuật Alpha-Beta cắt bỏ các nhánh vô dụng sớm hơn rất nhiều, tăng tốc độ tính toán.